In [1]:
import struct
import zlib

In [2]:
def imread_png_grayscale(filename):
    with open(filename, 'rb') as f:
        # Verify PNG signature
        signature = f.read(8)
        if signature != b'\x89PNG\r\n\x1a\n':
            raise ValueError("File is not a valid PNG image.")
        
        # Read chunks until we get the IHDR and IDAT chunks
        width = height = None
        bit_depth = color_type = None
        compressed_data = b''
        
        while True:
            # Each chunk has 4 parts: length (4 bytes), type (4 bytes), data, CRC (4 bytes)
            chunk_length = struct.unpack(">I", f.read(4))[0]  # Length of chunk data
            chunk_type = f.read(4)  # Type of chunk
            
            # Read the chunk data
            chunk_data = f.read(chunk_length)
            # Skip CRC
            f.read(4)
            
            if chunk_type == b'IHDR':
                # IHDR chunk contains metadata
                width, height, bit_depth, color_type, _, _, _ = struct.unpack(">IIBBBBB", chunk_data)
                if color_type != 0:
                    raise ValueError("Only grayscale PNG images are supported.")
                
            elif chunk_type == b'IDAT':
                # Collect IDAT data for decompression
                compressed_data += chunk_data
                
            elif chunk_type == b'IEND':
                # IEND marks the end of the file
                break
        
        # Decompress the IDAT data
        decompressed_data = zlib.decompress(compressed_data)
        
        # Convert decompressed data to a 2D array of pixels
        # Each row of the image starts with a filter byte
        image_data = []
        row_size = width
        
        i = 0
        for row in range(height):
            # Skip the filter byte
            filter_type = decompressed_data[i]
            i += 1
            
            # Read each pixel in the row
            row_data = []
            for col in range(row_size):
                row_data.append(decompressed_data[i])
                i += 1
            
            image_data.append(row_data)
        
    return image_data  # Returns a 2D list representing the grayscale image

In [5]:
def flatten_image(image_data):
    # Use a list comprehension to flatten the 2D array into a 1D array
    flattened_image = [pixel for row in image_data for pixel in row]
    return flattened_image

In [ ]:
import os
import random

# Directory path Switch this up for submission
directory_path = 'C:\\Users\\Admin\\coding-projs\\cmsc422\\HW\\422hw3\\facedata\\Face Data for Homework\\ATT'
sorted_files = sorted(os.listdir(directory_path))
label_and_data = [] # [label, data]

# Iterate through each file in the directory
for index, filename in enumerate(sorted_files):
    file_path = os.path.join(directory_path, filename)
    if 'README' in filename:
        continue

    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        # Find the part of the filename up to the first underscore
        class_number = filename.split('_', 1)[0]
        flattened_data = flatten_image(imread_png_grayscale(file_path))
        label_and_data.append([[class_number, flattened_data]])

random.shuffle(label_and_data)
print(len(label_and_data))

400


In [24]:
def euclidean_distance(img1, img2):
    # Ensure the images have the same length
    if len(img1) != len(img2):
        raise ValueError("Images must have the same size.")
    
    # Initialize the sum of squared differences
    squared_diff_sum = 0
    
    # Loop over each pixel value in the flattened arrays
    for i in range(len(img1)):
        # Calculate the difference between corresponding pixels
        diff = img1[i] - img2[i]
        
        # Square the difference and add to the sum
        squared_diff_sum += diff ** 2
    
    # Take the square root of the sum to get the Euclidean distance
    distance = squared_diff_sum ** 0.5
    return distance


In [28]:
five_folds = []
fold = []

for entry in label_and_data:
    if len(fold) == 79:
        fold.append(entry)
        five_folds.append([f for f in fold])
        fold = []
    else:
        fold.append(entry)
            

In [43]:
print(len(five_folds))

5


In [54]:
def get_1nn_predictions(train, test):
    amount_correct = 0
    for i in range(len(test)):
        test_me = test[i][0]
        # print(len(test_me))
        real_label = test_me[0]
        test_data = test_me[1]
        pred = [float('inf'), 'none']  # distance, label
        for j in range(len(train)):
            label = train[j][0][0]
            data = train[j][0][1]
            dist = euclidean_distance(data, test_data)
            if dist < pred[0]:
                pred = [dist, label]
        if pred[1] == real_label:
            amount_correct += 1
    print(f"Amount correct {amount_correct}")
    return amount_correct


In [55]:
accuracy  = []

for i in range(len(five_folds)):
    predict_for_me = five_folds[i]
    # print(len(predict_for_me))
    ammount_correct = 0
    for j in range(len(five_folds)):
        if i == j:
            continue
        test_on_me = five_folds[j]
        ammount_correct += get_1nn_predictions(test_on_me, predict_for_me)
    print(f"between fold {i} and {j} there were {ammount_correct} correct predictions")
    accuracy.append((ammount_correct / 320) * 100)

print( f"Total accuracy {sum(accuracy) / 5}" )
        

Amount correct 57
Amount correct 49
Amount correct 39
Amount correct 53
between fold 0 and 4 there were 198 correct predictions
Amount correct 53
Amount correct 58
Amount correct 45
Amount correct 48
between fold 1 and 4 there were 204 correct predictions
Amount correct 56
Amount correct 57
Amount correct 53
Amount correct 52
between fold 2 and 4 there were 218 correct predictions
Amount correct 56
Amount correct 43
Amount correct 52
Amount correct 50
between fold 3 and 4 there were 201 correct predictions
Amount correct 57
Amount correct 54
Amount correct 53
Amount correct 41
between fold 4 and 4 there were 205 correct predictions
Total accuracy 64.125


The printed statements here are misleading. That is not the number of correct predictions between fold i and j. It is the number of correct predictions with fold i as the test set

In [56]:
one_NN_accuracy = {sum(accuracy) / 5} 
print(f"one NN accuracy {one_NN_accuracy}")

one NN accuracy {64.125}
